# 2. Uproot - iteration and writing

<br><br><br><br><br>

## Managing work with Uproot.iterate and writing data to root files

In [ ]:
import numpy as np
import awkward as ak
import uproot
import hist

import IPython
import matplotlib.pyplot as plt
import matplotlib.pylab

<br><br><br>

### Get arrays in manageable chunks

The [iterate](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.HasBranches.html#iterate) method is like [arrays](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.HasBranches.html#arrays), but it can be used in a loop over chunks of the array.

How large are the chunks? You should set that with `step_size`.

In [ ]:
events = uproot.open("data/Zmumu.root")["events"]

In [ ]:
for arrays in events.iterate(step_size="50 kB"):   # 50 kB is very small! for illustrative purposes only!
    print(repr(arrays))

<br><br><br>

### Collections of files (like TChain)

If you want to read a bunch of files in one call, it has to be a function, rather than a method of [TTree](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html).

   * The equivalent of [TTree](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html) [arrays](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html#arrays) is [uproot.concatenate](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.concatenate.html). _(Reads everything at once: use this as a convenience on datasets you know are small!)_
   * The equivalent of [TTree](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html) [iterate](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TTree.TTree.html#iterate) is [uproot.iterate](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.iterate.html). _(This is the most useful one.)_
   * There's also an [uproot.lazy](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.lazy.html) _(More on this below.)_

In [ ]:
h1 = hist.Hist.new.Reg(500, 0, 500, name="mass").Double()
h3 = hist.Hist(
    hist.axis.Regular(250, 0, 500, name="mass", label="$m_{ll}$ [GeV]"),
    hist.axis.Integer(-2, 3, name="chargesum", label="$\Sigma \text{charge}_\mu$"),
    hist.axis.Variable([0, 0.2, 0.5, 1.0, 2.0, 5.0], name="deta", label="$\Delta\eta(m_1,m_2)$"),
    hist.storage.Double()
)
drawables = []

for muons in uproot.iterate(
    # filename(s)
    ["root://eospublic.cern.ch//eos/opendata/cms/derived-data/AOD2NanoAODOutreachTool/Run2012BC_DoubleMuParked_Muons.root:Events"],

    # branches to read as expressions
    ["Muon_pt", "Muon_eta", "Muon_phi", "Muon_charge"],

    # cut to apply to expressions
    cut="nMuon >= 2",

    # library dependent; for library="ak", try to split branch names at underscore and make nested records (poor man's NanoEvents)
    how="zip",

    # the all-important step_size!
    step_size="1 MB",

    # options you would normally pass to uproot.open
    xrootd_handler=uproot.MultithreadedXRootDSource,
    num_workers=10,
):
    # do everything you're going to do to this array
    os_cut = muons[:, "Muon", "charge", 0] != muons[:, "Muon", "charge", 1]
    mu1 = muons[os_cut, 0, "Muon"]
    mu2 = muons[os_cut, 1, "Muon"]
    
    # such as filling a histogram
    h1.fill(np.sqrt(2*mu1.pt*mu2.pt*(np.cosh(mu1.eta - mu2.eta) - np.cos(mu1.phi - mu2.phi))))
    
    # we can also look at same-sign events
    ss_cut = ~os_cut
    mu1 = muons[ss_cut, 0, "Muon"]
    mu2 = muons[ss_cut, 1, "Muon"]
    
    ss_invmass = np.sqrt(2*mu1.pt*mu2.pt*(np.cosh(mu1.eta - mu2.eta) - np.cos(mu1.phi - mu2.phi)))
    ss_deta = np.abs(mu1.eta - mu2.eta)
    ss_charge = mu1.charge + mu2.charge
    
    # filling a 3-dim histogram
    h3.fill(mass=ss_invmass, chargesum=ss_charge, deta=ss_deta)

    # a little magic to animate it; hardest part is removing the previous plots
    for drawable in drawables:
        drawable.stairs.remove()
        drawable.errorbar.remove()
        drawable.legend_artist.remove()
    drawables = h1.plot(color="blue")
    plt.yscale("log")
    IPython.display.display(matplotlib.pylab.gcf())
    IPython.display.clear_output(wait=True)

<br><br><br>

### Lazy arrays

Lazy arrays were introduced so that you can explore a large dataset without knowing ahead of time what parts you're going to read.

In [ ]:
events = uproot.lazy(
    # filename(s)
    ["root://eospublic.cern.ch//eos/opendata/cms/derived-data/AOD2NanoAODOutreachTool/Run2012BC_DoubleMuParked_Muons.root:Events"],
    # step_size is still important
    step_size="1 MB",
)
events

In [ ]:
events.type

In [ ]:
len(events)

It _looks like_ we've read that big remote file.

But actually (if we peek in the lazy array's cache), we've only read 3 TBaskets.

In [ ]:
events._caches[0].keys()

In [ ]:
events.Muon_pt

Now 4 TBaskets...

In [ ]:
events._caches[0].keys()

<div style="font-size: 25px; margin: 10px; padding: 30px; border: 1px dashed black;">
    Big important warning: <b>this access pattern is not read or memory efficient!</b>
</div>

This pattern is much better for exploration, to compute quantities without having to decide ahead of time which branches you'll need.

**Recommendation:** develop with lazy arrays, but then put the calculation into an [iterate](https://uproot.readthedocs.io/en/latest/uproot.behaviors.TBranch.iterate.html) loop or a [Coffea Processor](https://coffeateam.github.io/coffea/notebooks/processor.html).

<br><br><br>

### Writing histograms to root files

Writing histograms to root files is possible with a simple key-value syntax. np.histogram and hist/boost-histograms can be written without any wrapping code

[WritableDirectory](https://uproot.readthedocs.io/en/latest/uproot.writing.writable.WritableDirectory.html)

We use [uproot.recreate](https://uproot.readthedocs.io/en/latest/uproot.writing.writable.recreate.html) (which, like ROOT's "recreate" mode, will create or overwrite an existing file with the given name; [uproot.update](https://uproot.readthedocs.io/en/latest/uproot.writing.writable.update.html) is also available)

In [ ]:
with uproot.recreate("histos.root") as hfile:
    # we can write histograms in 3 dimensions (-> TH3D)
    hfile["threeD/hist"] = h3
    
    # project and save in 2 dimensions
    hfile["twoD/mass_deta"] = h3.project("mass", "deta")
    hfile["twoD/mass_charge"] = h3.project("mass", "chargesum")
    
    # slice and save the opposite-sign and same-sign invariant mass distributions
    hfile["oneD/mass_SS"] = h3.project("mass")
    hfile["oneD/mass_OS"] = h1

Lets open the file and check the contents

In [ ]:
hfo = uproot.open("histos.root")
hfo.keys(cycle=False)

How were our hist-histograms stored? 

As TH3, TH2, and TH1 types. 

In [ ]:
hfo.classnames()

We can convert back to hist and plot the two invariant mass projections, for same-sign and opposite-sign muon pairs:

In [ ]:
hfo["oneD/mass_SS"].to_hist().plot(), hfo["oneD/mass_OS"].to_hist().plot()
plt.yscale("log")

And here's the fraction of two-muon and two anti-muons events

In [ ]:
hfo["threeD/hist"].to_hist().project("chargesum").plot(density=True)

<br><br><br>

### Writing TTrees to root files

uproot supports writing multiple data types as TTrees, including numpy arrays and awkward arrays

[uproot.WritableTree](https://uproot.readthedocs.io/en/latest/uproot.writing.writable.WritableTree.html)

In [ ]:
# first we'll open a file
tree_file = uproot.recreate("tree.root")

for idx, muons in enumerate(uproot.iterate(
    # filename(s)
    ["root://eospublic.cern.ch//eos/opendata/cms/derived-data/AOD2NanoAODOutreachTool/Run2012BC_DoubleMuParked_Muons.root:Events"],
    # branches to read as expressions
    ["Muon_pt", "Muon_eta", "Muon_phi", "Muon_charge"],
    cut="nMuon >= 2",
    how="zip",
    step_size="10 MB",
    xrootd_handler=uproot.MultithreadedXRootDSource,
    num_workers=10,
    )):
    
        # OS muons again
        os_cut = muons[:, "Muon", "charge", 0] != muons[:, "Muon", "charge", 1]
        mu1 = muons[:, 0, "Muon"]
        mu2 = muons[:, 1, "Muon"]
        
        # calculate the invariant mass again
        invmass = np.sqrt(2*mu1.pt*mu2.pt*(np.cosh(mu1.eta - mu2.eta) - np.cos(mu1.phi - mu2.phi)))
        
        # compose two cuts to reduce data
        final_cut = os_cut & (invmass > 75)
        
        # lets create arrays with only the opposite-sign muons with invariant mass over 75
        final_muons = muons[final_cut].Muon
        final_mass = invmass[final_cut]
        
        # pack the arrays into a dictionary, keeping final_muons as an ak.array and final_mass as an np.array
        contents = {"Muon": final_muons, "InvMass": final_mass}
        
        # if this is the first write, assign directly 
        if idx == 0:
            tree_file["OSMuonEvents"] = contents
        # otherwise we extend
        else:
            tree_file["OSMuonEvents"].extend(contents)
tree_file.close()

What data types were stored?

In [ ]:
tf = uproot.open("tree.root")
tree = tf['OSMuonEvents']
tree.show()